In [ ]:
import os
import collections
import requests
from selenium import webdriver
from selenium.common.exceptions import NoSuchElementException
from selenium.common.exceptions import ElementNotInteractableException
from selenium.webdriver.support.ui import Select
import csv
import string
# import boto3

In [ ]:
def set_up_parkrun_dir():
    if not os.path.exists('./parkrun'):
        os.mkdir('./parkrun')

def scrape_personal_results(driver, athelete_number):
    
    column_headings = [
        'event',
        'event_url',
        'run_date',
        'run_date_url',
        'run_number',
        'run_number_url',
        'pos',
        'time',
        'age_grade',
        'pb'
    ]
    
    set_up_parkrun_dir()
    
    previous_event_urls = []
    try:
        with open('./parkrun/personal_results.csv', 'r') as personal_result:
            race_result_reader = csv.DictReader(personal_result, fieldnames=column_headings)
            for row in race_result_reader:
                previous_event_urls.append(row['run_number_url'])
    except FileNotFoundError:
        pass
        
    all_results_url = 'https://www.parkrun.org.uk/results/athleteeventresultshistory/?athleteNumber={}&eventNumber=0'
    driver.get(all_results_url.format(athelete_number))
    #//tagname[@attribute=’Value‘]
    tables = driver.find_elements_by_xpath("//table[@id='results']")
    run_rows = []
    for table in tables:
        if table.find_element_by_xpath(".//caption").text != 'All Results':
            pass
        else:
            run_rows = table.find_elements_by_xpath(".//tbody/tr[*]")
    with open('./parkrun/personal_results.csv', 'a+') as personal_results:  
        results_writer = csv.writer(personal_results)
        new_results = collections.deque()
        for row in run_rows:            
            run_number_url = row.find_element_by_xpath('td[3]/a').get_attribute('href')            
            if run_number_url in previous_event_urls:
                continue            
            data = [
                row.find_element_by_xpath('td[1]/a').text,
                row.find_element_by_xpath('td[1]/a').get_attribute('href'),
                row.find_element_by_xpath('td[2]/a').text,
                row.find_element_by_xpath('td[2]/a').get_attribute('href'),
                row.find_element_by_xpath('td[3]/a').text,
                run_number_url,
                row.find_element_by_xpath('td[4]').text,
                row.find_element_by_xpath('td[5]').text,
                row.find_element_by_xpath('td[6]').text,
                row.find_element_by_xpath('td[7]').text,
            ]
            new_results.appendleft(data)
        for data in new_results:
            results_writer.writerow(data)

In [ ]:
driver = webdriver.Firefox()
scrape_personal_results(driver,'1839227')

In [ ]:
def set_up_parkrun_race_results_dir():
    if not os.path.exists('./parkrun/race_results'):
        os.mkdir('./parkrun/race_results')
        
def close_search_tip_modal(driver):
    try:
        results_modal = driver.find_element_by_class_name('Results-modal-close')
        if results_modal:
            results_modal.click()
    except ElementNotInteractableException:
        pass
    
def scrape_results_for_event(event_name, event_number, results_url):
    set_up_parkrun_race_results_dir()
    race_result_filename = '{}_{}.csv'.format(event_name.lower().replace(' ', '_'), event_number)
    with open('./parkrun/race_results/{}'.format(race_result_filename), 'w+') as race_result:
        race_result_writer = csv.writer(race_result)
        driver.get(results_url)
        close_search_tip_modal(driver)
        select = Select(driver.find_element_by_name('display'))
        select.select_by_visible_text('Detailed')
        rows = driver.find_elements_by_xpath('//table/tbody/tr[*]')
        print(len(rows))
        for row in rows:
            data = []
            # position
            data.append(row.find_element_by_xpath('td[1]').text)
            # Name
            try:
                data.append(row.find_element_by_xpath('td[2]/div').text)
            except NoSuchElementException:
                data.append('')
            # Name detail
            try:
                data.append(row.find_element_by_xpath('td[2]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Gender
            try:
                data.append(row.find_element_by_xpath('td[3]/div').text)
            except NoSuchElementException:
                data.append('')
            # Gender detail
            try:
                data.append(row.find_element_by_xpath('td[3]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Age Group
            try:
                data.append(row.find_element_by_xpath('td[4]/div').text)
            except NoSuchElementException:
                data.append('')
            # Age Group detail
            try:
                data.append(row.find_element_by_xpath('td[4]/div[2]').text)
            except NoSuchElementException:
                data.append('')
            # Club
            try:
                data.append(row.find_element_by_xpath('td[5]/div').text)  
            except NoSuchElementException:
                data.append('')
            # Time
            try: 
                data.append(row.find_element_by_xpath('td[6]/div[1]').text)
            except NoSuchElementException:
                data.append('')
            # Time detail
            try: 
                data.append(row.find_element_by_xpath('td[6]/div[2]').text)
            except NoSuchElementException:
                data.append('')  
            race_result_writer.writerow(data)
                    
def scrape_new_race_results(driver):
    with open('./parkrun/personal_results.csv', 'r') as personal_results:
        personal_results_reader = csv.reader(personal_results)
        for result in personal_results_reader:
            race_result_filename = '{}_{}.csv'.format(result[0].lower().replace(' ', '_'), result[4])
            if not os.path.exists('./parkrun/race_results/{}'.format(race_result_filename)):
                scrape_results_for_event(result[0], result[4], result[3])
    
def scrape_all_race_results(driver):
    with open('./parkrun/personal_results.csv', 'r') as personal_results:
        personal_results_reader = csv.reader(personal_results)
        for result in personal_results_reader:
            scrape_results_for_event(result[0], result[4], result[3])
            
        

In [ ]:
driver = webdriver.Firefox()
scrape_new_race_results(driver)

In [ ]:
session = boto3.Session(profile_name='personal')
dynamodb_client = session.client('dynamodb')

In [ ]:
def update_personal_results_database(client):
    with open('./parkrun/personal_results.csv', 'r') as personal_results_file:
        personal_results_reader = csv.reader(personal_results_file)
        for row in personal_results_reader:
            response = dynamodb_client.put_item(
                TableName = 'personal-results',
                Item = {
                    'date' : {
                        'S': row[2]
                    },
                    'event-name': {
                        'S': row[0]
                    },
                    'event_url': {
                        'S': row[1]
                    },
                    'run_date_url': {
                        'S': row[3]
                    },
                    'run_number': {
                        'N': row[4]
                    },
                    'run_number_url': {
                        'S': row[5]
                    },
                    'pos': {
                        'N': row[6]
                    },
                    'time': {
                        'S': row[7]
                    },
                    'age_grade': {
                        'N': row[8].strip('%')
                    },
                    'pb': {
                        'BOOL': True if row[9].strip() == 'PB' else False
                    }               

                }

            )
        
        

In [ ]:
update_personal_results_database(dynamodb_client)

In [ ]:
def update_race_results_database(client, table_name = 'race-results'):
    race_results_filenames = {}
    with open('./parkrun/personal_results.csv', 'r') as personal_results_file:
        personal_results_reader = csv.reader(personal_results_file)
        for row in personal_results_reader:
            event_name = row[0].lower().replace(' ', '_')
            event_number= row[4]
            race_result_filename = '{}_{}.csv'.format(event_name, event_number)
            personal_results_reader = csv.reader(personal_results_file)
            race_results_filenames[race_result_filename] = {'event_name': event_name, 'event_number': event_number}
        
    for race_result_filename in race_results_filenames:
        with open('./parkrun/race_results/{}'.format(race_result_filename), 'r') as race_result:
            race_result_reader = csv.reader(race_result)
            row_num = 0
            
            item_batch = {table_name : []}
            
            for row in race_result_reader:
                print("Processing row {} of file {}".format(row_num, race_result_filename))
                row_num += 1
                
                item_mapping = {   
                    'event_name_number': {'type': 'S', 'data':'{}_{}'.format(race_results_filenames[race_result_filename]['event_name'], race_results_filenames[race_result_filename]['event_number'])},
                    'position': {'type': 'N', 'data':row[0]},
                    'name': {'type': 'S', 'data':row[1]},
                    'name_detail': {'type': 'S', 'data':row[2]},
                    'gender': {'type': 'S', 'data':row[3]},
                    'gender_detail': {'type': 'S', 'data':row[4]},
                    'age_group': {'type': 'S', 'data':row[5]},
                    'age_group_detail': {'type': 'S', 'data':row[6]},
                    'club': {'type': 'S', 'data':row[7]},
                    'time': {'type': 'S', 'data':row[8]},
                    'time_detail': {'type': 'S', 'data':row[9]},
                }
                
                item = {}
                
                for mapping in item_mapping:
                    if item_mapping[mapping]['data'] != '':
                        item[mapping] = {item_mapping[mapping]['type']: item_mapping[mapping]['data']}
                
                item_batch[table_name].append({'PutRequest': {'Item': item}})
                
                if len(item_batch[table_name]) == 25:
                    response = client.batch_write_item(RequestItems = item_batch)
                    if response['UnprocessedItems']:
                        item_batch = response['']
                    else:
                        item_batch = {table_name : []}
            #debug
                    print(response)
                
                
                
                



In [ ]:
update_race_results_database(dynamodb_client, 'batch-test-table')

In [ ]:
import psycopg2
from psycopg2.extensions import ISOLATION_LEVEL_AUTOCOMMIT

In [ ]:
conn = psycopg2.connect(user="postgres", password="postgres", host="172.19.0.2", dbname="parkrundb")
conn.set_isolation_level(ISOLATION_LEVEL_AUTOCOMMIT)

In [ ]:
create_db_sql = 'CREATE DATABASE prdb;'
cursor = conn.cursor()
cursor.execute(create_db_sql)

In [ ]:
cursor = conn.cursor()
create_table_sql = """
DROP TABLE IF EXISTS race_results;

CREATE TABLE race_results (
    id serial PRIMARY KEY,
    event_name varchar, 
    event_number integer,
    position integer,
    name varchar,
    name_detail varchar,
    gender varchar,
    gender_detail varchar,
    age_group varchar,
    age_group_detail varchar,
    club varchar,
    time varchar,
    time_detail varchar,
    UNIQUE(event_name, event_number, position)
);
"""
response = cursor.execute(create_table_sql)
print(response)

In [ ]:
def update_race_results_postgres(cursor):
    race_results_filenames = {}
    with open('./parkrun/personal_results.csv', 'r') as personal_results_file:
        personal_results_reader = csv.reader(personal_results_file)
        for row in personal_results_reader:
            event_name = row[0].lower().replace(' ', '_')
            event_number= row[4]
            race_result_filename = '{}_{}.csv'.format(event_name, event_number)
            personal_results_reader = csv.reader(personal_results_file)
            race_results_filenames[race_result_filename] = {'event_name': event_name, 'event_number': event_number}
        
    for race_result_filename in race_results_filenames:
        with open('./parkrun/race_results/{}'.format(race_result_filename), 'r') as race_result:
            race_result_reader = csv.reader(race_result)
            row_num = 0
                       
            for row in race_result_reader:
                print("Processing row {} of file {}".format(row_num, race_result_filename))
                row_num += 1
                
                name = (row[1])
                
                insert_row_sql = """
                    INSERT INTO race_results (
                        event_name, 
                        event_number,
                        position,
                        name,
                        name_detail,
                        gender,
                        gender_detail,
                        age_group,
                        age_group_detail,
                        club,
                        time,
                        time_detail
                        )
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s);"""
                
                insert_row_data = (
                        race_results_filenames[race_result_filename]['event_name'].replace('_', ' '),
                        race_results_filenames[race_result_filename]['event_number'],
                        row[0],
                        string.capwords(row[1], ' '),
                        row[2],
                        row[3],
                        row[4],
                        row[5],
                        row[6],
                        row[7],
                        row[8],
                        row[9]
                )

                response = cursor.execute(insert_row_sql, insert_row_data)

In [ ]:
update_race_results_postgres(cursor)